## Problem Set 5: PDA

_csc427, semester 212
<br>
university of miami
<br>
date: 10 march 2021
<br>
update: 10 march 2021_

---

### Student name: Jeffrey Hudak

---


### Non-determinism and PDA's.

The class MachineModelPDA implements a PDA. It uses the Schrödinger approach of multible worlds computation. In this case, the state that is traced includes three elements: the location in the input string, the state, and the complete stack. I call this the _extended state_.

The simulation consists of iterating over a "bag" of such in-progress computations, replacing each as they are reviewed with the possible continuations.

It is possible that such computations do not halt, but do not strictly loop. For instance, the "loop" of pushing the same symbol onto the stack does generate new extended states, so is not looping. 

It is clear to our minds that such a computation path can be terminated. It is a limitation of the machine model that it, itself, cannot do this pruning, and is the nature of the _intrinsic non-determinism_ of Context Free Languages. 

For instance, Context Free Languages are __not__ closed under complementation. If they were the perhaps a companion calculation on the complement language will over-rule these loops, announcing definitively that the string is not in the original lanuage, and all hunts should end.

However, in your program, for full credit, your programs must halt!

It is possible to write a conceptually correct program that accepts but does not halt on reject. For a string not in the language, the language defintion, being non-deterministic, premits that the machine run forever loving for the accepting computation that it will never find.

Obviously, we do not write programs in this mindset. Spell checkers do not run forever looking for some dictionary in some dictionary to justify some obvious mispelling. Although I understand Scrabble games often end in such disputes.



## PDA Description Format


__PDA Description:__

    A PDA description is a dictionary with,
    
        'states': a list of states.
        'alphabet': a list of word letters
        'symbols': a list of stack symbols
        'transitions': a dictionary of
                tuple(state,alphabet_e,symbols_e):list(tuple((state,symbols_e)))
        'start': a state (the start state)
        'accept': a list of states (the accepting states)
  
    A state is a string
    A word letter is a string of length 1, but not the string ':'
    A stack symbol is a string but not the string ':'
    alphabet_e is alphabet adjoining ':'
    symbols_e is the symbols adjoining ':'
    The string ':' is used in transtitions to denote the empty string.


In [1]:
"""
The verbose switch:
    Set this true or false, to run code verbosely
"""

verbose = False

In [2]:
class MachineModelPDA:
    """
    A machine description is a dictionary with,
        'states': a list of states.
        'alphabet': a list of word letters
        'symbols': a list of stack symbols
        'transitions': a dictionary of
                tuple(state,alphabet_e,symbols_e):list(tuple((state,symbols_e)))
        'start': a state (the start state)
        'accept': a list of states (the accepting states)
  
    a state is a string
    a word letter is a string of length 1, but not the string ':'
    a stack symbol is a string but not the string ':'
    alphabet_e is alphabet adjoining ':'
    symbols_e is the symbols adjoining ':'
    
    and extended state (ext_state) is the tuple (state,location,stack contents)
    the stack contents is a tuple of symbols, with the top of the stack at index 0,
    
    the current state is a list of extended states, each extended state the result of 
    computing up to but not including the index in the word given by the integer "location" .
    
    """
    
    def __init__(self,machine_description):
        
        def check_transitions():
            ## check that it is a dict((state,alapha_e,symbols_e):list((state_symbols_e)))
            assert type(self.transitions)==type({})
            for k in self.transitions:
                if k[0] not in self.states:
                    print(f'state {k[0]} not a state')
                    assert False
                if not( k[1]==':' or k[1] in self.alphabet ):
                    print(f'letter {k[1]} not in alphabet')
                    assert False
                if not( k[2]==':' or k[2] in self.symbols ):
                    print(f'symbol {k[2]} not a symbol')
                    assert False
                vs = self.transitions[k]
                assert type(vs)==type([])
                for v in vs:
                    if v[0] not in self.states:
                        print(f'state {v[0]} not a state')
                        assert False
                    if not( v[1]==':' or v[1] in self.symbols):
                        print(f'symbol {v[1]} not a symbol')
                        assert False
        
        def check_alphabet():
            ## the alphabet symbols must be single characters,
            ## and should not be the : character which has the special
            ## meaning of a possible non-input consuming transition.
            for a in self.alphabet:
                assert len(a)==1 and a != ':'
                
        def check_start():
            assert self.start_state in self.states
            
        def check_accept_states():
            assert type(self.accept_states)==type([])
            for s in self.accept_states:
                assert s in self.states
        
        self.states = machine_description['states']
        self.alphabet = machine_description['alphabet']
        self.symbols = machine_description['symbols']
        self.transitions = machine_description['transitions']
        self.start_state = machine_description['start'] 
        self.accept_states = machine_description['accept']
        
        check_transitions()
        check_alphabet()
        check_start()
        check_accept_states()
        
        # for the non-deterministic trace we need to have the complete state, 
        # including stack contents and current input location
        self.current_states = [(self.start_state,0,())]
        
        # hint list. this works for verification
        self.merlin = []


    def do_transition(self,word,current_state):
        
        def trans_aux(letter,new_i,state,symbol,stack):
            new_states = []
            key = (state,letter,symbol)
            if key in self.transitions:
                for new_state,new_symbol in self.transitions[key]:
                    if new_symbol != ':':
                        nstack = (new_symbol,)+stack
                    else:
                        nstack = stack
                    new_states.append((new_state,new_i,nstack))
            return new_states
            
        state, i, stack = current_state
        new_states = []
        
        # transition of input letter
        if i<len(word):
            if len(stack)>0:
                new_states.extend(trans_aux(word[i],i+1,state,stack[0],stack[1:]))
            new_states.extend(trans_aux(word[i],i+1,state,':',stack))

        # epsilon transitions
        if len(stack)>0:
            new_states.extend(trans_aux(':',i,state,stack[0],stack[1:]))
        new_states.extend(trans_aux(':',i,state,':',stack))
        
        return set(new_states)  # glean duplicates of possible states
        
    def approximate_compute(self,word,limit):
        
        ## returns True if word is in language, False if it is not,
        ## and None if the computation was terminated and did not
        ## determine membership.
        
        def word_accepted():
            for ext_state in self.current_states:
                state, i, stack = ext_state
                if i==len(word) and state in self.accept_states:
                    return True
            return False
        
        self.current_states = set([(self.start_state,0,())])
        for steps in range(limit):
            
            if verbose: print(self.current_states)
            if word_accepted():
                return True
            if len(self.current_states)==0:
                return False

            new_states = set()
            for ext_state in self.current_states:
                new_states |= self.do_transition(word,ext_state)
            self.current_states = new_states
            if (len(self.current_states)>limit*limit):
                return None
            
        return None   # computation abandoned
        
        
    def describe(self,name=""):
        print("Machine Description:",name)
        print("\tstates:",len(self.states))
        print("\t\t",self.states)
        print("\tsymbols:",len(self.symbols))
        print("\t\t",self.symbols)
        print("\ttransitions:",len(self.transitions))
        for t,v in self.transitions.items():
            print(f"\t\t{t}  ->  {v}")
        print("\taccept states:",len(self.accept_states))
        print("\t\t",self.accept_states)
        print()


def test_machine(pda_description,test_cases,name="",limit=100):
    
    print('running test:',name)
    pda = MachineModelPDA(pda_description)
    if verbose: pda.describe(name)
    for (t,r) in (test_cases):
        if pda.approximate_compute(t,limit) != r:
            print(r,'\t|'+t+'|','\tWRONG, ABORT')
            return False
        print(r,'\t|'+t+'|','\tOK')
    print("** passes test")
    return True
  

## Exercise A

From Sipser Exercises 2.5. Give a description of a PDA accepting a string over $\{\,0,1\,\}$ when,

1. it has at least 3 ones;
1. it begins and ends with the same symbol;
1. it is of odd length;
1. it is of odd length and the middle symbol is a 0;
1. it is a palindrome.
1. never. The language is the  empty set.


In [3]:
pda_2_5_1 = {
    'states':['s', 'q0', 'q1', 'q2', 'q3'],
    'alphabet':['0', '1'],
    'symbols':[],
    'transitions':{
        ('s', ':', ':'):[('q0', ':')],
        ('q0', '1', ':'):[('q1', ':')], ('q0', '0', ':'):[('q0', ':')],
        ('q1', '1', ':'):[('q2', ':')], ('q1', '0', ':'):[('q1', ':')],
        ('q2', '1', ':'):[('q3', ':')], ('q2', '0', ':'):[('q2', ':')],
        ('q3', '1', ':'):[('q3', ':')], ('q3', '0', ':'):[('q3', ':')],
    },
    'start':'s',
    'accept':['q3']
}

test_2_5_1 = [
    ('', False),
    ('11',False),
    ('111',True),
    ('1111',True),
    ('00001101', True),
    ('100001', False),
    ('10101', True),
    ('000011100000000000', True),
    ('00001100000000000', False),
    ('100010001', True),
    ('10001000100', True),
    ('10000011', True),
    ('100000001', False),
]

pda_2_5_2 = {
    'states':['s', 'q0', 'q1', 'q2', 'q3', 'qf'],
    'alphabet':['0', '1'],
    'symbols':['$', 'Z', 'W'],
    'transitions':{
        ('s', ':', ':'):[('q0', '$')],
        ('q0', '1', ':'):[('q3', 'W'), ('q2', ':')], ('q0', '0', ':'):[('q1', 'Z'), ('q2', ':')],
        ('q1', '1', ':'):[('q1', ':')],              ('q1', '0', 'Z'):[('q2', ':'), ('q1', 'Z')],
        ('q3', '1', 'W'):[('q2', ':'), ('q3', 'W')], ('q3', '0', ':'):[('q3', ':')],
        ('q2', ':', '$'):[('qf', ':')],
        
    },
    'start':'s',
    'accept':['qf']  
}

test_2_5_2 = [
    ('',False),
    ('1',True),
    ('0',True),
    ('01',False),
    ('101',True),
    ('10101', True),
    ('01010', True),
    ('0111110', True),
    ('01111010101010101010101010001111010', True),
    ('10001', True),
    ('1000', False),
    ('1000010', False),
    ('0111101', False),
    ('01010101010101', False),
    ('10011', True),
    ('0110010000100', True),
    ('100000110', False)
]

pda_2_5_3 = {
    'states':['s', 'q0', 'q1'],
    'alphabet':['0', '1'],
    'symbols':[],
    'transitions':{
        ('s', ':', ':'):[('q0', ':')],
        ('q0', '0', ':'):[('q1', ':')], ('q0', '1', ':'):[('q1', ':')],
        ('q1', '0', ':'):[('q0', ':')], ('q1', '1', ':'):[('q0', ':')],
    },
    'start':'s',
    'accept':['q1']    
}

test_2_5_3 = [
    ('',False),
    ('10',False),
    ('101',True),
    ('111', True),
    ('1111', False),
    ('000', True),
    ('0000', False),
    ('0110', False),
    ('01010', True),
    ('01110', True),
    ('10001', True),
    ('10101', True),
    ('010101', False),
    ('011110', False),
    ('100001', False),
    ('101101', False),
    ('010010', False)
]

pda_2_5_4 = {
    'states':['s', 'q0', 'qL', 'qf'],
    'alphabet':['0', '1'],
    'symbols':['$', 'J'],
    'transitions':{
        ('s', ':', ':'):[('q0', '$')],
        ('q0', '0', ':'):[('qL', ':'), ('q0', 'J')], ('q0', '1', ':'):[('q0', 'J')],
        ('qL', '0', 'J'):[('qL', ':')],              ('qL', '1', 'J'):[('qL', ':')],
        ('qL', ':', '$'):[('qf', ':')],
    },
    'start':'s',
    'accept':['qf']   
}



test_2_5_4 = [
    ('',False),
    ('101',True),
    ('111',False),
    ('001',True),
    ('11001', True),
    ('000', True),
    ('11111', False),
    ('11011', True),
    ('01010', True),
    ('11101', False),
    ('1110111', True),
    ('0000000', True),
    ('0010110', True),
    ('1101101', False),
    ('0000', False),
    ('000000', False),
    ('110011', False),
]

# tricky because even or odd
pda_2_5_5 = {
    'states':['s','q0','q1','qf'],
    'alphabet':['0','1'],
    'symbols':['$','Z','W'],
    'transitions':{
        ('s',':',':'):[('q0','$')],
        
        ('q0','0',':'):[('q0','Z'), ('q1', ':')], ('q0','1',':'):[('q0','W'), ('q1', ':')],
        ('q0',':',':'):[('q1', ':')],
        
        ('q1','0','Z'):[('q1',':')], ('q1','1','W'):[('q1',':')],
        ('q1',':','$'):[('qf',':')],
    },
    'start':'s',
    'accept':['qf', 's']  
}

test_2_5_5 = [
    ('',True),
    ('10', False),
    ('01', False),
    ('1',True),
    ('00',True),
    ('10',False),
    ('11',True),
    ('101', True),
    ('00',True),
    ('0110',True),
    ('1001',True),
    ('11000011',True),
    ('1100110011',True),
    ('1',True),
    ('0',True),
    ('1100',False),
    ('0011',False),
    ('0101',False),
    ('1010',False),
    ('00011001',False),
    ('1111111',True),
    ('1110111', True),
    ('101101', True),
    ('11000100011', True),
    ('111011', False),
    ('0101011101010101010010101', False),
    ('01010111011101010', True),
    ('000000000', True),
    ('0000000000', True),
    ('10111101', True),
]

# not to be confused with the language {':'}
pda_2_5_6 = {
    'states':['q0', 'q1'],
    'alphabet':['0', '1'],
    'symbols':[],
    'transitions':{
    },
    'start':'q0',
    'accept':['q1'],    
}

test_2_5_6 = [
    ('',False),
    ('1',False),
    ('00',False),
    ('010',False),
    (':', False)
]

### Basic Tests

In [4]:

def basic_test(pdas,tests):
    i = 0
    correct = 0
    for mac, test in zip(pdas,tests):
        if test_machine(mac,test,name='exercise '+str(i+1),limit=100):
            correct += 1
        i += 1
    print(f'\n{correct} out of {i} correct')


In [5]:
exer_A = [
    pda_2_5_1,
    pda_2_5_2,
    pda_2_5_3,
    pda_2_5_4,
    pda_2_5_5,
    pda_2_5_6,
]

test_A = [
    test_2_5_1,
    test_2_5_2,
    test_2_5_3,
    test_2_5_4,
    test_2_5_5,
    test_2_5_6,
]

basic_test(exer_A,test_A)

running test: exercise 1
False 	|| 	OK
False 	|11| 	OK
True 	|111| 	OK
True 	|1111| 	OK
True 	|00001101| 	OK
False 	|100001| 	OK
True 	|10101| 	OK
True 	|000011100000000000| 	OK
False 	|00001100000000000| 	OK
True 	|100010001| 	OK
True 	|10001000100| 	OK
True 	|10000011| 	OK
False 	|100000001| 	OK
** passes test
running test: exercise 2
False 	|| 	OK
True 	|1| 	OK
True 	|0| 	OK
False 	|01| 	OK
True 	|101| 	OK
True 	|10101| 	OK
True 	|01010| 	OK
True 	|0111110| 	OK
True 	|01111010101010101010101010001111010| 	OK
True 	|10001| 	OK
False 	|1000| 	OK
False 	|1000010| 	OK
False 	|0111101| 	OK
False 	|01010101010101| 	OK
True 	|10011| 	OK
True 	|0110010000100| 	OK
False 	|100000110| 	OK
** passes test
running test: exercise 3
False 	|| 	OK
False 	|10| 	OK
True 	|101| 	OK
True 	|111| 	OK
False 	|1111| 	OK
True 	|000| 	OK
False 	|0000| 	OK
False 	|0110| 	OK
True 	|01010| 	OK
True 	|01110| 	OK
True 	|10001| 	OK
True 	|10101| 	OK
False 	|010101| 	OK
False 	|011110| 	OK
False 	|100001| 	OK
False 

## Exercise B

From Sipser Exercises 2.7. Give a description of a PDA accepting strings over $\{\,a,b,\}$
such that,

1. the string has more a's than b's, 
1. it is the complement of the a^n b^n, n &ge; 0,
3. they are of the form w#x, with the string reverse(w) a substring of x, and '#' only where shown,
4. they are of the form x_1#x_2#...#x_k, for k &ge;1, and j, x_i and x_j are reverses of each other, and '#''s only where shown, 
   
   
   

In [6]:
pda_2_7_1 = {
    'states':['s', 'q0', 'q1', 'q2', 'q3', 'q4', 'qf'],
    'alphabet':['a', 'b'],
    'symbols':['$', 'P', 'N'],
    'transitions':{
        ('s', ':', ':'):[('q0', '$')],
        ('q0', 'a', ':'):[('q1', 'P'), ('qf', ':')], ('q0', 'b', ':'):[('q3', 'N')],
        
        ('q1', 'a', ':'):[('q1', 'P')], ('q3', 'b', ':'):[('q3', 'N')],
        ('q1', 'b', 'P'):[('q4', ':')], ('q3', 'a', 'N'):[('q2', ':')],
        ('q1', ':', 'P'):[('qf', ':')],
        
        ('q2', 'a', 'N'):[('q2', ':')], ('q4', 'b', 'P'):[('q4', ':')],
        ('q2', ':', '$'):[('s', ':')],  ('q4', ':', '$'):[('s', ':')],
        ('q2', 'b', ':'):[('q3', 'N')], ('q4', 'a', ':'):[('q1', 'P'), ('qf', ':')],
        ('q2', ':', 'P'):[('qf', ':')], ('q4', ':', 'P'):[('qf', ':')],
        # This may be unnecessary ^^
        
    },
    'start':'s',
    'accept':['qf']    
}

pda_2_7_2 = {
    'states':['s', 'q0', 'q1', 'q2', 'q3', 'q4', 'wrong', 'tempwrong', 'qf'],
    'alphabet':['a', 'b'],
    'symbols':['$', 'P', 'N'],
    'transitions':{
        ('s', ':', ':'):[('q0', '$')],
        ('q0', 'a', ':'):[('q1', 'P')], ('q0', 'b', ':'):[('q3', 'N')],
        
        ('q1', 'a', ':'):[('q1', 'P')], ('q3', 'b', ':'):[('q3', 'N')],
        ('q1', 'b', 'P'):[('q4', ':')], ('q3', 'a', ':'):[('wrong', ':')],
        
        ('q4', 'b', 'P'):[('q4', ':')],
        ('q4', ':', '$'):[('qf', ':')],
        ('q4', 'a', ':'):[('wrong', ':')],
        ('q4', ':', 'P'):[('tempwrong', ':')],
        
        ('qf', 'a', ':'):[('wrong', ':')], ('qf', 'b', ':'):[('wrong', ':')],
        
        ('wrong', 'a', ':'):[('wrong', ':')], ('wrong', 'b', ':'):[('wrong', ':')]
    },
    'start':'s',
    'accept':['q1', 'q3', 'wrong', 'tempwrong']
}

pda_2_7_3 = {
    'states':['s', 'q0', 'q1', 'q2', 'q3', 'qf'],
    'alphabet':['a', 'b', '#'],
    'symbols':['$', 'A', 'B'],
    'transitions':{ #   aba#bbbababbb
        ('s', ':', ':'):[('q0', '$')],
        
        ('q0','a',':'):[('q0','A')], ('q0','b',':'):[('q0','B')],
        ('q0','#',':'):[('q2',':'), ('q3',':')],
        
        ('q2','a','A'):[('q2',':')], ('q2','b','B'):[('q2',':')],
        ('q2',':','$'):[('qf',':')],
        
        ('q3','a',':'):[('q3',':'), ('q2',':')], ('q3','b',':'):[('q3',':'), ('q2',':')],
        
        ('qf','a',':'):[('qf',':')], ('qf','b',':'):[('qf',':')]
    },
    'start':'s',
    'accept':['qf']
}

pda_2_7_4 = {
    'states':['s', 'q0', 'q1', 'q2', 'q3', 'q4', 'q5', 'qf', 'qp'],
    'alphabet':['a', 'b', "#"],
    'symbols':['$', 'A', 'B'],
    'transitions':{
        # Start with empty, send to q0
        ('s', ':', ':'):[('q0', '$'), ('q4', '$')],
        
        # q0 accumulates all a's and b's before a hashtag.
        # sends the stack to q3, and begins a new stack via going so s to grab another $
        ('q0', 'a', ':'):[('q0', 'A')], ('q0', 'b', ':'):[('q0', 'B')],
        ('q0', '#', ':'):[('q2', ':'), ('q1', ':'), ('s', ':')],
        
        # Tries to decompile the stack by removing a's and b's
        # If it finds a $, then we found a reverse and can be labeled as temporarily True
        ('q1', 'a', 'A'):[('q1', ':')], ('q1', 'b', 'B'):[('q1', ':')],
        ('q1', ':', '$'):[('qp', ':')],
        
        # Keeps the stack given to it safe and sends the saved stack to q1 for testing after each new string (#)
        ('q2', 'a', ':'):[('q2', ':')], ('q2', 'b', ':'):[('q2', ':')],
        ('q2', '#', ':'):[('q1', ':'), ('q2', ':')],
        
        #q4 and q5 are from the solving palindrome from above. If it is true, go to temp 'palindrome' state
        ('q4','a',':'):[('q4','A'), ('q5', ':')], ('q4','b',':'):[('q4','B'), ('q5', ':')],
        ('q4',':',':'):[('q5',':')],
        
        ('q5','a','A'):[('q5',':')], ('q5','b','B'):[('q5',':')],
        ('q5',':','$'):[('qp',':')],
        
        #If we are in the 'reverse passed' state and we find a #, then it can be permanently true
        ('qp','#',':'):[('qf',':')],
        
        ('qf', 'a', ':'):[('qf', ':')], ('qf', 'b', ':'):[('qf', ':')], ('qf', '#', ':'):[('qf', ':')],
    },
    'start':'s',
    'accept':['qf', 'qp']
}

# the string has more a's than b's,
test_2_7_1 =[
    ('a',True),
    ('b',False),
    ('ab',False),
    ('aba',True),
    ('aaa', True),
    ('bbaa', False),
    ('bbaaa', True),
    ('bababaa', True),
    ('aaabbbb', False),
    ('aaabbbba', False),
    ('aaaaaaabbbabb', True),
    ('bbbbbaaaaa', False),
    ('bbbbaaaaab', False),
    ('abababaa', True),
    ('ababab', False),
    ('aaabbbaa', True),
    ('aaabb', True),
    ('', False)
]

# it is the complement of the a^n b^n, n ≥ 0,
test_2_7_2 =[
    ('ab',False),
    ('ba',True),
    ('abab',True),
    ('b', True),
    ('abaaaa', True),
    ('aaaaab', True),
    ('bbbab', True),
    ('aaabbb', False),
    ('aabba', True),
    ('aaabbba', True),
    ('aaaabbbbbbb', True),
    ('bbbbbbbb', True),
    ('aaaaa', True),
    ('', False),
    ('aab', True),
    ('baaabbb', True),
    ('abb', True),
    ('aaab', True),
    ('aaaaabbbb', True),
    ('aaaabbbbb', True),
    ('aaaaabbbbb', False),
    ('abbbbbbbbbbbbbbbb', True),
    ('aaaaaaaaaaaaaab', True),
    ('aaaaaaabbbbbbb', False),
    ('aaaaaabbbbbbbbbbbbbbbba', True),
    ('aba', True)
]

# they are of the form w#x, with the string reverse(w) a substring of x, and '#' only where shown,
test_2_7_3 =[
    ('#',True),
    ('a#a',True),
    ('ab#ab',False),
    ('ab#aba',True),
    ('a#a#a',False),
    ('aaa#aaa', True),
    ('aba#bab', False),
    ('aabba#aaaaaaabbaa', True),
    ('bbb#abb', False),
    ('abaa#abaa', False),
    ('abaab#aabaabaaaaa', True),
    ('abaab#aabaabbaaaaa', False),
    ('aab#abaa#', False),
    ('aa#aa#', False),
    ('a#bbbba', True),
    ('a#bbbba#', False),
    ('#', True),
    ('ababababa#abababababaa', True),
    ('ababbababa#abababababaa', False),
    ('ababababa#ababababab', True),
    ('ababababa##abababababaa', False),
    ('aababababa#abababababaa', True),
]

# they are of the form x_1#x_2#...#x_k, for k ≥1, and j, x_i and x_j are reverses of each other, and '#''s only where shown,
test_2_7_4 =[
    ('ab#aa#ab',True),
    ('abb#bba',True),
    ('ba#baa#aab#ba',True),
    ('ba#ba',False),
    ('ba##ba',True),
    ('ab#ba', True),
    ('ab#a#bb', True),
    ('a', True),
    ('##', True),
    ('bbbbaaa#bbab#aaba#aaabbbb', True),
    ('bbaaa#ba#aab#ab#', True),
    ('bbaaa#ba#aab#ab', True),
    ('aab#ba###aaabb#bba##bbab', True),
    ('aab#ba#aaabb#bba##bbab', True),
    ('aab##ba#aaabb#bba#bbab', True),
    ('aab#ba#aaabb##bba#bbab', True),
    ('aab#ba#aaabb#bba###bbab', True),
    ('aab###ba###aaabb#bba#bbab', True),
    ('aab#ba#aaabb###bba#bbab', True),
    ('aab#ba###aaabb#bba#bbab', True),
    ('aab#ba#aaabb#bba#bbab', False),
    ('ab#ba#aaabb#bba#bbab', True),
    ('aab#babb#aaabb#bba#bbab', True),
    ('abb#bbaa',False),
    ('', True),
    
]


In [7]:
exer_B = [
    pda_2_7_1,
    pda_2_7_2,
    pda_2_7_3,
    pda_2_7_4,
]

test_B = [
    test_2_7_1,
    test_2_7_2,
    test_2_7_3,
    test_2_7_4,
]

basic_test(exer_B,test_B)

running test: exercise 1
True 	|a| 	OK
False 	|b| 	OK
False 	|ab| 	OK
True 	|aba| 	OK
True 	|aaa| 	OK
False 	|bbaa| 	OK
True 	|bbaaa| 	OK
True 	|bababaa| 	OK
False 	|aaabbbb| 	OK
False 	|aaabbbba| 	OK
True 	|aaaaaaabbbabb| 	OK
False 	|bbbbbaaaaa| 	OK
False 	|bbbbaaaaab| 	OK
True 	|abababaa| 	OK
False 	|ababab| 	OK
True 	|aaabbbaa| 	OK
True 	|aaabb| 	OK
False 	|| 	OK
** passes test
running test: exercise 2
False 	|ab| 	OK
True 	|ba| 	OK
True 	|abab| 	OK
True 	|b| 	OK
True 	|abaaaa| 	OK
True 	|aaaaab| 	OK
True 	|bbbab| 	OK
False 	|aaabbb| 	OK
True 	|aabba| 	OK
True 	|aaabbba| 	OK
True 	|aaaabbbbbbb| 	OK
True 	|bbbbbbbb| 	OK
True 	|aaaaa| 	OK
False 	|| 	OK
True 	|aab| 	OK
True 	|baaabbb| 	OK
True 	|abb| 	OK
True 	|aaab| 	OK
True 	|aaaaabbbb| 	OK
True 	|aaaabbbbb| 	OK
False 	|aaaaabbbbb| 	OK
True 	|abbbbbbbbbbbbbbbb| 	OK
True 	|aaaaaaaaaaaaaab| 	OK
False 	|aaaaaaabbbbbbb| 	OK
True 	|aaaaaabbbbbbbbbbbbbbbba| 	OK
True 	|aba| 	OK
** passes test
running test: exercise 3
True 	|#| 	OK
True 	|a#

# 